In [1]:
import numpy as np
import theano as theano
import theano.tensor as T
import time
import operator
from utils import *
from lstm_theano import *
from gru_theano import *

In [4]:
# Load data
from utils import load_and_proprocess_data
VOCABULARY_SIZE = 8000
X_train, y_train, word_to_index, index_to_word = load_and_proprocess_data(VOCABULARY_SIZE)

Reading CSV file...
Parsed 79170 sentences.
Found 65751 unique words tokens.
Using vocabulary size 2000.
The least frequent word in our vocabulary is 'status' and appeared 58 times.


In [2]:
# Do a gradient check
np.random.seed(0)
model = GRUTheano(100, 10)
gradient_check_theano(model, [0,1,2,3], [1,2,3,4])

Performing gradient check for parameter U with size 3000.
Gradient check for parameter U passed.
Performing gradient check for parameter W with size 300.
Gradient check for parameter W passed.
Performing gradient check for parameter b with size 30.
Gradient check for parameter b passed.
Performing gradient check for parameter V with size 1000.
Gradient check for parameter V passed.
Performing gradient check for parameter b2 with size 100.
Gradient check for parameter b2 passed.


/Users/dennybritz/projects/wildml/rnn-tutorial-lstm/venv/lib/python2.7/site-packages/theano/scan_module/scan_perform_ext.py:133: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility
  from scan_perform.scan_perform import *


In [10]:
# Load parameters of pre-trained model
# model = load_model_parameters_theano('data/pretrained.npz')

# Build model and train

LEARNING_RATE = 1e-4
NEPOCH = 50
HIDDEN_DIM = 80

model = GRUTheano(VOCABULARY_SIZE, hidden_dim=HIDDEN_DIM)

t1 = time.time()
model.sgd_step(X_train[10], y_train[10], LEARNING_RATE)
t2 = time.time()
print "SGD Step time: %f milliseconds" % ((t2 - t1) * 1000.)

train_with_sgd(model, X_train[:500], y_train[:500], LEARNING_RATE, NEPOCH, evaluate_loss_after=1)
# save_model_parameters_theano('data/mymodel.npz', model)

SGD Step time: 51.363945 milliseconds
2015-10-14 11:57:16: Loss after num_examples_seen=0 epoch=0: 7.600784
2015-10-14 11:57:42: Loss after num_examples_seen=500 epoch=1: 7.478756
2015-10-14 11:58:07: Loss after num_examples_seen=1000 epoch=2: 7.239769
2015-10-14 11:58:35: Loss after num_examples_seen=1500 epoch=3: 6.802384
2015-10-14 11:59:02: Loss after num_examples_seen=2000 epoch=4: 6.674400
2015-10-14 11:59:30: Loss after num_examples_seen=2500 epoch=5: 6.574331


KeyboardInterrupt: 

In [11]:
sentence_start_token = "SENTENCE_START"
sentence_end_token = "SENTENCE_END"
unknown_token = "UNKNOWN_TOKEN"
    
def generate_sentence(model):
    # We start the sentence with the start token
    new_sentence = [word_to_index[sentence_start_token]]
    # Repeat until we get an end token
    while not new_sentence[-1] == word_to_index[sentence_end_token]:
        next_word_probs = model.forward_propagation(new_sentence)
        sampled_word = word_to_index[unknown_token]
        # We don't want to sample unknown words
        while sampled_word == word_to_index[unknown_token]:
            samples = np.random.multinomial(1, next_word_probs[-1])
            sampled_word = np.argmax(samples)
        new_sentence.append(sampled_word)
    sentence_str = [index_to_word[x] for x in new_sentence[1:-1]]
    return sentence_str
 
num_sentences = 10
senten_min_length = 7
 
for i in range(num_sentences):
    sent = []
    # We want long sentences, not sentences with one or two words
    while len(sent) < senten_min_length:
        sent = generate_sentence(model)
    print " ".join(sent)

stuff drink most //www.reddit.com/message/compose/ hit content talked .
out different impact stupid realized yesterday . plans issues decided
us cash road watching rank hurt being
bot experience agree fan . facts it property
incredibly places game wise . first .
ride carry with . saw unique opportunity . performed old .
humans spend weapons worked . increase delete .
pm link email look starts . choice animals
list ^^ love to=tweetposter *i within fighting .
9 let shoot accurate = fill while .
